In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
ques_df=pd.read_pickle("Questions_dataset.pkl")
ques_df.head()

,question1,question2
108281,How can I get more information on cryptocurrency?,What can you say about kerala Congress's exit?
365075,How much does each miner in Gold Rush: Alaska ...,What kind of dog does the Hoffman crew have wi...
399424,Can a stock exchange be listed in another stoc...,What is stock exchange and its functions?
116651,What is the top speed of the quickest football...,Who are your top 5 best foreign football playe...
251097,Is there any way of knowing if a guy is gay?,What are some subtle ways to tell if a guy is ...


In [4]:
feature_df=pd.read_pickle("Featured_dataset.pkl")
feature_df.head()

,is_duplicate,q1_len,q2_len,q1_num_of_words,q2_num_of_words,word_common,word_total,words_share
108281,0,49,46,8,8,1,16,0.062
365075,0,98,79,19,17,2,32,0.062
399424,0,57,41,10,7,2,16,0.125
116651,0,55,89,10,16,3,25,0.120
251097,1,44,50,11,12,5,22,0.227


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# merge texts

questions=list(ques_df['question1']) + (list(ques_df['question2']))
cv=CountVectorizer(max_features=3000)
q1_arr,q2_arr =np.vsplit(cv.fit_transform(questions).toarray(),2)

In [6]:
temp_df1=pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2=pd.DataFrame(q2_arr, index=ques_df.index)
temp_df=pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(30000, 6000)

In [7]:
feature_df=pd.concat([feature_df,temp_df], axis=1)
print(feature_df.shape,"\n")
feature_df.head(3)

(30000, 6008) 



,is_duplicate,q1_len,q2_len,q1_num_of_words,q2_num_of_words,word_common,word_total,words_share,0,1,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
108281,0,49,46,8,8,1,16,0.062,0,0,...,1,0,0,0,0,0,0,0,0,0
365075,0,98,79,19,17,2,32,0.062,0,0,...,0,0,0,0,0,0,0,0,0,0
399424,0,57,41,10,7,2,16,0.125,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(feature_df.iloc[:,1:].values,
                                               feature_df.iloc[:,0].values,
                                               test_size=0.2,
                                               random_state=42
                                              )

In [9]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((24000, 6007), (6000, 6007), (24000,), (6000,))

In [10]:
# Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf=RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)
accuracy_score(y_test,y_pred)

0.7595

In [16]:
# XGBoost Classifier
from xgboost import XGBClassifier

xgb=XGBClassifier()
xgb.fit(x_train,y_train)
y_pred=xgb.predict(x_test)
accuracy_score(y_test,y_pred)

0.7571666666666667

In [15]:
# Lightgbm Classifier

from lightgbm import LGBMClassifier

lgbm=LGBMClassifier()
lgbm.fit(x_train,y_train)
y_pred=lgbm.predict(x_test)
accuracy_score(y_test,y_pred)

[LightGBM] [Info] Number of positive: 8726, number of negative: 15274
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.436047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7975
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 2657
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.363583 -> initscore=-0.559845
[LightGBM] [Info] Start training from score -0.559845


0.7663333333333333

    Example: 
Question 1: "How do I make money?"

Question 2: "What are the best ways to make money online as a student?"


Tokens Q1: {"How", "do", "I", "make", "money"} (Length=5)

Tokens Q2: {"What", "are", "the", "best", "ways", "to", "make", "money", "online", "as", "a", "student"} (Length=12)


Stopwords Q1: {"how", "do", "i"} (Length = 3)

Stopwords Q2: {"what", "are", "the", "to", "as", "a"} (Length = 6)


Words Q1: {"make", "money"}  (Length=2)

Words Q2: {"best", "ways", "make", "money", "online", "student"} (Length=6)


Common Words: {"make", "money"} (Count = 2)

Common StopWords: 0


    TOKEN FEATURES


1. Common Word count (min)=
   no. of common words/(min(words(q1,q2)))
   2/min(2,6) = 2/2 = 1
2. Common Word count (max)
   no. of common words/(max(words(q1,q2)))
   2/min(2,6) = 2/6 = 0.333
3. Common Stopword count (min)
   no of common stopwords/(min(stopwords(q1,q2)))
   0/min(3,6) = 0
4. Common Stopword count (max)
   no of common stopwords/(max(stopwords(q1,q2)))
   0/max(3,6) = 0
5. Common Token count (min)
   no of common tokens/(min(tokens(q1,q2)))
   2/(min(5,12)) = 2/5 = 0.4
6. Common Token count (max)
    no of common tokens/(max(tokens(q1,q2)))
    2/(max(5,12)) = 2/12 = 0.16
7. Last Word equal
   Is last word Q1 == last word Q2?
   "money" == "student"
   = 0
8. First Word equal
   Is first word Q1 == first word Q2?
   "how" == "what"
   = 0

    LENGTH BASED FEATURES
1. mean_len = (len(q1)+len(q2))/2 = 5+12/2 =8.5
2. abs_len_diff = |5-12| = 7
3. congest_substr_ratio = len(longest_common_substring)/(min(len(q1),len(q2))+1)

Q1: "How to learn Python?"

Q2: "What is the best way to learn Python quickly?"

The longest common part is "learn Python" = 12 characters
q1 is shorter with 20 characters
congest_substr_ratio = 12/20 = 0.6